# Networks: structure, evolution & processes
**Internet Analytics - Lab 2**

---

**Group:** *P*

**Names:**

* *Matthias Leroy*
* *Pierre Fouche*
* *Alexandre Poussard*

---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

---

## 2.3 Epdemics

#### Exercise 2.9: Simulate an epidemic outbreak

In [ ]:
import epidemics_helper
from networkx.readwrite import json_graph
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np


filename = '../data/nyc_augmented_network.json'
json_load = json.load(open(filename))
G = json_graph.node_link_graph(json_load)

sir = epidemics_helper.SimulationSIR(G, beta=10.0, gamma=0.1)
sir.launch_epidemic(source=23654, max_time=100.0)

In [ ]:
def plotStatus(graph,epidemic):
    nbOfNodes = nx.number_of_nodes(graph)
    susTime = []
    infTime = []
    recTime = []
    timeInf60 = 0
    timeRec60 = 0
    for i in range(0,101):
        countSus = 0
        countInf = 0
        countRec = 0
        for j in range(nbOfNodes):
            if epidemic.inf_time[j] <= i and epidemic.rec_time[j] >= i:
                countInf += 1
            elif epidemic.inf_time[j] > i:
                countSus += 1
            elif epidemic.rec_time[j] <= i:
                countRec += 1
        percInf = (countInf*100)/nbOfNodes
        percSus = (countSus*100)/nbOfNodes
        percRec = (countRec*100)/nbOfNodes
        if percInf >= 60 and timeInf60 == 0:
            timeInf60 = i
        elif percRec >= 60 and timeRec60 == 0:
            timeRec60 = i
        
        infTime.append(percInf)
        susTime.append(percSus)
        recTime.append(percRec)
        
    plt.plot(infTime)
    plt.plot(susTime)
    plt.plot(recTime)
    plt.show()

    print("at the day", timeInf60, ", 60% of the population is infected")
    print("at the day", timeRec60, ", 60% of the population is recovered")


In [ ]:
plotStatus(G,sir)

In [ ]:
def colorNodes(time):
    color = []
    for i in range(nbOfNodes):
        if sir.inf_time[i] <= time and sir.rec_time[i] > time:
            color.append('r')
        elif sir.rec_time[i] <= time:
            color.append('b')
        elif sir.inf_time[i] > time:
            color.append('g')
    return color

coordinatesDic = {}
colorAt1 = colorNodes(1)
colorAt3 = colorNodes(3)
colorAt30 = colorNodes(30)

for i in json_load["nodes"]:
    coordinatesDic[i["id"]] = (i["coordinates"][0],i["coordinates"][1])

#G.add_nodes_from(coordinatesDic.keys())
nx.draw_networkx(G, coordinatesDic, with_labels=False, node_size=10, node_color=colorAt1)
plt.show()
plt.axis('off');

---

### 2.3.1 Stop the apocalypse!

#### Exercise 2.10: Strategy 1

In [ ]:
graph2 = json_graph.node_link_graph(json_load)
for i in range(1000):
    index = np.random.randint(0,nx.number_of_edges(graph2))
    graph2.remove_edge(graph2.edges()[index][0],graph2.edges()[index][1])

startingNode = np.random.randint(0,nx.number_of_nodes(graph2))
sir = epidemics_helper.SimulationSIR(graph2, beta=10.0, gamma=0.1)
sir.launch_epidemic(source=startingNode, max_time=100.0)
plotStatus(graph2,sir)

#### Exercise 2.11: Strategy 2